In [6]:
!pip install brian2


In [7]:

from brian2 import *
import numpy as np
import pandas as pd
import os

# ---------------- CONFIG ---------------- #
defaultclock.dt = 0.025*ms
SIM_TIME = 300*ms
I_STIM = 0.8*nA
N_SAMPLES_PER_CHANNEL = 150
DOWNSAMPLE = 10   # same idea as cardiac notebook

SAVE_DIR = "pain_dataset"
os.makedirs(SAVE_DIR, exist_ok=True)

CHANNELS = [
    "Na_fast","Na_p","K_dr","K_A",
    "Ca","HCN","TRPV1","Leak",
    "Bg1","Bg2"
]

# ---------------- HH MODEL ---------------- #
eqs = '''
dv/dt = (
    I_stim
    - gNa*m**3*h*(v-ENa)
    - gNap*p*(v-ENa)
    - gK*n**4*(v-EK)
    - gKA*a**3*b*(v-EK)
    - gCa*(v-ECa)
    - gHCN*(v-EH)
    - gTRP*(v-Etrp)
    - gLeak*(v-EL)
    - gBg1*(v-EL)
    - gBg2*(v-EL)
) / Cm : volt

dm/dt = alpha_m*(1-m)-beta_m*m : 1
dh/dt = alpha_h*(1-h)-beta_h*h : 1
dn/dt = alpha_n*(1-n)-beta_n*n : 1

dp/dt = (p_inf-p)/5/ms : 1
da/dt = (a_inf-a)/3/ms : 1
db/dt = (b_inf-b)/8/ms : 1

p_inf = 1/(1+exp(-(v+55*mV)/(6*mV))) : 1
a_inf = 1/(1+exp(-(v+50*mV)/(20*mV))) : 1
b_inf = 1/(1+exp((v+80*mV)/(6*mV))) : 1

alpha_m = 0.1/mV*(v+40*mV)/(1-exp(-(v+40*mV)/(10*mV)))/ms : Hz
beta_m = 4*exp(-(v+65*mV)/(18*mV))/ms : Hz
alpha_h = 0.07*exp(-(v+65*mV)/(20*mV))/ms : Hz
beta_h = 1/(1+exp(-(v+35*mV)/(10*mV)))/ms : Hz
alpha_n = 0.01/mV*(v+55*mV)/(1-exp(-(v+55*mV)/(10*mV)))/ms : Hz
beta_n = 0.125*exp(-(v+65*mV)/(80*mV))/ms : Hz

gNa : siemens
gNap : siemens
gK : siemens
gKA : siemens
gCa : siemens
gHCN : siemens
gTRP : siemens
gLeak : siemens
gBg1 : siemens
gBg2 : siemens

I_stim : amp

ENa : volt (constant)
EK : volt (constant)
ECa : volt (constant)
EL : volt (constant)
EH : volt (constant)
Etrp : volt (constant)
Cm : farad (constant)
'''

def simulate_trace(scale_channel=None, scale=1.0):
    neuron = NeuronGroup(1, eqs, method='exponential_euler')

    # Constants
    neuron.ENa = 50*mV
    neuron.EK = -77*mV
    neuron.ECa = 120*mV
    neuron.EL = -54.4*mV
    neuron.EH = -30*mV
    neuron.Etrp = 0*mV
    neuron.Cm = 1*uF

    # Baseline conductances
    neuron.gNa = 120*msiemens
    neuron.gNap = 5*msiemens
    neuron.gK = 36*msiemens
    neuron.gKA = 10*msiemens
    neuron.gCa = 2*msiemens
    neuron.gHCN = 1*msiemens
    neuron.gTRP = 0.8*msiemens
    neuron.gLeak = 0.3*msiemens
    neuron.gBg1 = 0.2*msiemens
    neuron.gBg2 = 0.15*msiemens

    neuron.I_stim = I_STIM
    neuron.v = -65*mV

    if scale_channel:
        setattr(neuron, scale_channel, getattr(neuron, scale_channel) * scale)

    M = StateMonitor(neuron, 'v', record=True)

    # Explicitly create and run a Network for this simulation
    net = Network(neuron, M)
    net.run(SIM_TIME)

    return np.array(M.v[0] / mV)

# ---------------- DATA GENERATION ---------------- #
rows = []

for label, ch in enumerate(CHANNELS):
    print("Generating:", ch)

    for _ in range(N_SAMPLES_PER_CHANNEL):
        baseline = simulate_trace()
        scale = np.random.uniform(0.6, 1.4)

        mapping = {
            "Na_fast": "gNa",
            "Na_p": "gNap",
            "K_dr": "gK",
            "K_A": "gKA",
            "Ca": "gCa",
            "HCN": "gHCN",
            "TRPV1": "gTRP",
            "Leak": "gLeak",
            "Bg1": "gBg1",
            "Bg2": "gBg2"
        }

        perturbed = simulate_trace(mapping[ch], scale)

        delta_ap = (perturbed - baseline)[::DOWNSAMPLE]

        rows.append(np.concatenate([[label], delta_ap]))

# ---------------- SAVE CSV ---------------- #
cols = ["label"] + [f"dV_{i}" for i in range(len(rows[0]) - 1)]
df = pd.DataFrame(rows, columns=cols)

csv_path = os.path.join(SAVE_DIR, "pain_ap_difference_dataset.csv")
df.to_csv(csv_path, index=False)

print("Saved:", csv_path)
print("Shape:", df.shape)


Generating: Na_fast
Generating: Na_p
Generating: K_dr
Generating: K_A
Generating: Ca
Generating: HCN
Generating: TRPV1
Generating: Leak
Generating: Bg1
Generating: Bg2
Saved: pain_dataset/pain_ap_difference_dataset.csv
Shape: (1500, 1201)


In [8]:
from google.colab import files
files.download('pain_dataset/pain_ap_difference_dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>